# Project Title
### Data Engineering Capstone Project

#### Project Summary
- This project aims to collect and find data sources to use for analytics tables, app back-end, and source-of-truth databases.
- The data will be checked for quality issues and necessary cleaning steps will be noted. The next steps will involve creating a data model and pipelines, while ensuring that the data quality checks and integrity constraints are in place.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, substring, udf, sum
from pyspark.sql.types import *

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark=create_spark_session()

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project, I aim to analyze and model four different datasets to prepare them for end use cases such as analytics tables, app back-end, and source-of-truth databases. I will identify and gather data from at least two sources, which include immigration data to the United States, airport codes, demographic data of U.S. cities, and temperature data. I will explore and assess the data to identify any data quality issues, document necessary cleaning steps, and define a data model. The data model will be pipelined with the data and ETL will be run to create the data pipelines and the data model. Additionally, I will ensure data quality checks, integrity constraints, and perform unit tests for the scripts to ensure that they are functioning correctly.

#### Describe and Gather Data 
The four datasets used in this project are:

***Immigration data to the United States (df_Immigration_sample)***

***Temperature data (df_temp)***

***Demographic data of U.S. cities (df_demographics)***

***Airport codes (df_airport_codes)***

The immigration data is a sample dataset that includes information on immigrants to the United States, including their country of origin, mode of transportation, and arrival date. The temperature data contains information on the temperatures of various cities around the world. The demographic data of U.S. cities includes information on the racial and ethnic makeup of cities in the United States. The airport codes dataset includes information on airport codes and their corresponding cities and countries. All datasets were obtained from publicly available sources.

In [3]:
# Read in the data here :
df_Immigration_sample = pd.read_csv('./immigration_data_sample.csv')
df_temp = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
df_demographics = pd.read_csv("./us-cities-demographics.csv", delimiter=";")
df_airport_codes = pd.read_csv("./airport-codes_csv.csv")

#### Printing the head of the first dataset df_i94:

In [4]:
df_Immigration_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [5]:
df_Immigration_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

#### Describing the first dataset df_i94:

In [6]:
df_Immigration_sample.describe()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,dtadfile,entdepu,biryear,insnum,admnum
count,1.000000e+03,1.000000e+03,1000.0,1000.0,1000.000000,1000.00000,1000.000000,1000.000000,951.000000,1000.000000,1000.000000,1000.0,1.000000e+03,0.0,1000.000000,35.000000,1.000000e+03
mean,1.542097e+06,3.040461e+06,2016.0,4.0,302.928000,298.26200,20559.680000,1.078000,20575.037855,42.382000,1.859000,1.0,2.016042e+07,NaN,1973.618000,3826.857143,6.937237e+10
std,9.152879e+05,1.799818e+06,0.0,0.0,206.485285,202.12039,8.995027,0.485955,24.211234,17.903424,0.386353,0.0,4.951657e+01,NaN,17.903424,221.742583,2.338134e+10
min,1.092500e+04,1.320800e+04,2016.0,4.0,103.000000,103.00000,20545.000000,1.000000,20547.000000,1.000000,1.000000,1.0,2.016040e+07,NaN,1923.000000,3468.000000,0.000000e+00
25%,7.214422e+05,1.412170e+06,2016.0,4.0,135.000000,131.00000,20552.000000,1.000000,20561.000000,30.750000,2.000000,1.0,2.016041e+07,NaN,1961.000000,3668.000000,5.599301e+10
50%,1.494568e+06,2.941176e+06,2016.0,4.0,213.000000,213.00000,20560.000000,1.000000,20570.000000,42.000000,2.000000,1.0,2.016042e+07,NaN,1974.000000,3887.000000,5.931477e+10
75%,2.360901e+06,4.694151e+06,2016.0,4.0,438.000000,438.00000,20567.250000,1.000000,20580.000000,55.000000,2.000000,1.0,2.016042e+07,NaN,1985.250000,3943.000000,9.343623e+10
max,3.095749e+06,6.061994e+06,2016.0,4.0,746.000000,696.00000,20574.000000,9.000000,20715.000000,93.000000,3.000000,1.0,2.016080e+07,NaN,2015.000000,4686.000000,9.502151e+10


In [7]:
full_df_immigration = pd.read_sas('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', 'sas7bdat', encoding="ISO-8859-1")

In [8]:
full_df_immigration.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [9]:
full_df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [10]:
full_df_immigration.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3.096313e+06,3.096074e+06,2.953856e+06,3.095511e+06,3.096313e+06,3096313.0,3.095511e+06,3.096313e+06
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,2.055985e+04,1.073690e+00,2.057395e+04,4.176761e+01,1.845393e+00,1.0,1.974232e+03,7.082885e+10
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,8.777339e+00,5.158963e-01,2.935697e+01,1.742026e+01,3.983910e-01,0.0,1.742026e+01,2.215442e+10
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,2.054500e+04,1.000000e+00,1.517600e+04,-3.000000e+00,1.000000e+00,1.0,1.902000e+03,0.000000e+00
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,2.055200e+04,1.000000e+00,2.056100e+04,3.000000e+01,2.000000e+00,1.0,1.962000e+03,5.603523e+10
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,2.056000e+04,1.000000e+00,2.057000e+04,4.100000e+01,2.000000e+00,1.0,1.975000e+03,5.936094e+10
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,2.056700e+04,1.000000e+00,2.057900e+04,5.400000e+01,2.000000e+00,1.0,1.986000e+03,9.350987e+10
max,6.102785e+06,2016.0,4.0,9.990000e+02,7.600000e+02,2.057400e+04,9.000000e+00,4.542700e+04,1.140000e+02,3.000000e+00,1.0,2.019000e+03,9.991557e+10


In [11]:
full_df_immigration.to_csv('./Cleaned Data/full_immigration.csv', index=False)

#### Printing the head of the second dataset df_temp:

In [12]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [13]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB



#### Describing the second dataset df_temp:

In [14]:
df_temp.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,8.235082e+06,8.235082e+06
mean,1.672743e+01,1.028575e+00
std,1.035344e+01,1.129733e+00
min,-4.270400e+01,3.400000e-02
25%,1.029900e+01,3.370000e-01
50%,1.883100e+01,5.910000e-01
75%,2.521000e+01,1.349000e+00
max,3.965100e+01,1.539600e+01


#### Printing the head of the third dataset df_demographics:

In [15]:
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [16]:
df_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


#### Describing the third dataset df_demographics:

In [17]:
df_demographics.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


#### Printing the head of the fourth dataset df_airport_codes:

In [18]:
df_airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [19]:
df_airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


#### Describing the fourth dataset df_airport_codes:

In [20]:
df_airport_codes.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Cleaning The First Dataset Cleaning:

In [21]:
full_immigration_df = spark.read.format('csv').option('header', 'true').load('./Cleaned Data/full_immigration.csv')
full_immigration_df.createOrReplaceTempView("immigration_table")

In [22]:
full_immigration_df.printSchema()

root
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: string (nullable = 

In [23]:
full_immigration_df.show(10)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1897628485.0| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SEO| n

In [24]:
# check the total number of records of full_immigration_df
full_immigration_df.count()

3096313

In [25]:
# check for cicid if it unique or not and if unique I will make it a primary key:
from pyspark.sql.functions import countDistinct

full_immigration_df.select(countDistinct("cicid")).show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              3096313|
+---------------------+



In [26]:
### Adding a new column 'arrival_date' to the immigration dataset
full_immigration_df = spark.sql("""
    SELECT *, date_add(to_date('1960-01-01'), arrdate) AS arrival_date 
    FROM immigration_table
""")


# Adding a new column 'departure_date' to the immigration dataset
full_immigration_df = spark.sql("""
    SELECT *, 
        CASE 
            WHEN depdate >= 1.0 THEN date_add(to_date('1960-01-01'), depdate)
            WHEN depdate IS NULL THEN NULL
            ELSE 'N/A' 
        END AS departure_date 
    FROM (
        SELECT *, date_add(to_date('1960-01-01'), arrdate) AS arrival_date 
        FROM immigration_table
    )
""")

### Displaying the first five rows of the updated full immigration dataset
full_immigration_df.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_date|departure_date|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1897628485.0| null|      B2|  2016-04-29|          null|
|  7

In [27]:
# Filtering the immigration dataset to only include rows where 'departure_date_2' is greater than or equal to 'arrival_date'
full_immigration_df = full_immigration_df.filter(full_immigration_df.departure_date >= full_immigration_df.arrival_date)

# Displaying the first 5 rows of the filtered immigration dataset
full_immigration_df.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_date|departure_date|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    null| null|      T|      O|   null|      M| 1961.0|09302016|     M|  null|     OS|  666643185.0|   93|      B2|  2016-04-01|    2016-08-25|
| 16

In [28]:
print(f"Number of rows: {full_immigration_df.count()}")

Number of rows: 2953481


In [29]:
# Counting the number of rows for each unique value in the 'i94mode' column
mode_counts = spark.sql("""
    SELECT i94mode, COUNT(*) AS count
    FROM immigration_table
    GROUP BY i94mode
""")

# Displaying the result
mode_counts.show()

+-------+-------+
|i94mode|  count|
+-------+-------+
|    1.0|2994505|
|   null|    239|
|    9.0|   8560|
|    2.0|  26349|
|    3.0|  66660|
+-------+-------+



In [30]:
# Defining a dictionary to map i94mode values to arrival modes
arrival_modes = {1.0: 'Air', 2.0: 'Sea', 3.0: 'Land'}

# Adding a new column 'arrival_mode' to the immigration dataset using a UDF
def map_arrival_mode(i94mode):
    return arrival_modes.get(i94mode, 'Not_reported')

map_arrival_mode_udf = udf(map_arrival_mode, StringType())

# Adding the 'arrival_mode' column to the 'full_immigration_df' DataFrame
full_immigration_df = full_immigration_df.withColumn('arrival_mode', map_arrival_mode_udf('i94mode'))

# Displaying the first row of the updated immigration dataset
full_immigration_df.show()

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_date|departure_date|arrival_mode|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    null| null|      T|      O|   null|      M| 1961.0|09302016|     M|  null|     OS|  666643185.0|   93|   

In [31]:
print(f"Number of rows: {full_immigration_df.count()}")

Number of rows: 2953481


In [32]:
# Counting the number of rows for each unique value in the 'i94visa' column
visa_counts = spark.sql("""
    SELECT i94visa, COUNT(*) AS count
    FROM immigration_table
    GROUP BY i94visa
""")

# Displaying the result
visa_counts.show()

+-------+-------+
|i94visa|  count|
+-------+-------+
|    1.0| 522079|
|    2.0|2530868|
|    3.0|  43366|
+-------+-------+



In [33]:
# Defining a dictionary to map i94visa values to visa types
visa_types = {1.0: 'Business', 2.0: 'Pleasure', 3.0: 'Student'}

def map_visa_type(i94visa):
    return visa_types.get(i94visa, 'Not_reported')

map_visa_type_udf = udf(map_visa_type, StringType())

# Adding the 'visa_type' column to the 'immigration_table' DataFrame
full_immigration_df = full_immigration_df\
    .withColumn('visa_type', map_visa_type_udf('i94visa'))

# Displaying the first row of the updated immigration dataset
full_immigration_df.show()

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_date|departure_date|arrival_mode|   visa_type|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+------------+
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    null| null|      T|      O|   null|      M| 1961.0|09302016|     M

In [34]:
print(f"Number of rows: {full_immigration_df.count()}")

Number of rows: 2953481


In [35]:
# Using the DataFrame API to group the immigration data by gender and count the number of records
gender_counts = spark.sql("""
    SELECT gender, COUNT(*) AS count
    FROM immigration_table
    GROUP BY gender
""")
# Displaying the gender counts
gender_counts.show()

+------+-------+
|gender|  count|
+------+-------+
|     F|1302743|
|  null| 414269|
|     M|1377224|
|     U|    467|
|     X|   1610|
+------+-------+



In [36]:
def clean_full_immigration_df_gender(full_immigration_df):
    # Filtering the immigration data by gender using the DataFrame API
    full_immigration_df = full_immigration_df \
        .filter(full_immigration_df.gender.isin(['M', 'F']))

    # Creating a temporary view for the filtered data
    full_immigration_df.createOrReplaceTempView('immigration_table')

    # return the cleaned data:
    return full_immigration_df
    
full_immigration_df = clean_full_immigration_df_gender(full_immigration_df)

# Displaying the first row of the filtered data
full_immigration_df.show()

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_date|departure_date|arrival_mode|   visa_type|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+------------+--------------+------------+------------+
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    null| null|      T|      O|   null|      M| 1961.0|09302016|     M

In [37]:
print(f"Number of rows: {full_immigration_df.count()}")

Number of rows: 2544951


In [38]:
def clean_full_immigration_df(full_immigration_df):    
    # Selecting and renaming columns from the immigration data using the DataFrame API
    cols = ['cicid', 'year', 'month', 'immigrant_birth_country', 'immigrant_residence_country', 'adm_port',
            'arrival_date', 'departure_date', 'arrival_mode', 'arrival_state', 'age', 'visa_type', 'count',
            'birth_year', 'gender', 'airline', 'admission_num', 'flight_num']
    full_immigration_df = full_immigration_df.select(
        col('cicid'), col('i94yr').alias('year'), col('i94mon').alias('month'),
        col('i94cit').alias('immigrant_birth_country'), col('i94res').alias('immigrant_residence_country'),
        col('i94port').alias('adm_port'), col('arrival_date'), col('departure_date'), col('arrival_mode'),
        col('i94addr').alias('arrival_state'), col('i94bir').alias('age'), col('visa_type'), col('count'),
        col('biryear').alias('birth_year'), col('gender'), col('airline'), col('admnum').alias('admission_num'),
        col('fltno').alias('flight_num')
    ).toDF(*cols)
    # return the cleaned data:
    return full_immigration_df
    
full_immigration_df = clean_full_immigration_df(full_immigration_df)
# Displaying the selected columns from the immigration data
full_immigration_df.show(10)

+-----+------+-----+-----------------------+---------------------------+--------+------------+--------------+------------+-------------+----+------------+-----+----------+------+-------+-------------+----------+
|cicid|  year|month|immigrant_birth_country|immigrant_residence_country|adm_port|arrival_date|departure_date|arrival_mode|arrival_state| age|   visa_type|count|birth_year|gender|airline|admission_num|flight_num|
+-----+------+-----+-----------------------+---------------------------+--------+------------+--------------+------------+-------------+----+------------+-----+----------+------+-------+-------------+----------+
| 15.0|2016.0|  4.0|                  101.0|                      101.0|     WAS|  2016-04-01|    2016-08-25|Not_reported|           MI|55.0|Not_reported|  1.0|    1961.0|     M|     OS|  666643185.0|        93|
| 27.0|2016.0|  4.0|                  101.0|                      101.0|     BOS|  2016-04-01|    2016-04-05|Not_reported|           MA|58.0|Not_reporte

In [39]:
print(f"Number of rows: {full_immigration_df.count()}")

Number of rows: 2544951


In [40]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [41]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [42]:
df_temp.isnull().sum()

dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64

In [43]:
df_temp.duplicated().sum()

0

In [44]:
def clean_df_temp(df_temp):
    # Remove rows with missing temperature values
    df_temp = df_temp.dropna()

    # filter the  for the United States to make it useful with our immigration data
    df_temp = df_temp[df_temp['Country']=='United States']

    # Convert dt to datetime
    df_temp.dt = pd.to_datetime(df_temp.dt)
    # return the cleaned data:
    return df_temp

df_temp = clean_df_temp(df_temp)
df_temp.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W
47560,1820-06-01,25.682,2.008,Abilene,United States,32.95N,100.53W
47561,1820-07-01,26.268,1.802,Abilene,United States,32.95N,100.53W
47562,1820-08-01,25.048,1.895,Abilene,United States,32.95N,100.53W
47563,1820-09-01,22.435,2.216,Abilene,United States,32.95N,100.53W
47564,1820-10-01,15.830,2.169,Abilene,United States,32.95N,100.53W


In [45]:
df_temp.to_csv('./Cleaned Data/df_temp.csv', index=False)

### Cleaning the Third dataset:

In [46]:
def clean_df_demographics(df_demographics):
    # Remove rows with missing values
    df_demographics = df_demographics.dropna().reset_index(drop=True)

    # Convert numeric columns to appropriate data types
    numeric_columns = ["Median Age", "Male Population", "Female Population", "Total Population", 
                       "Number of Veterans", "Foreign-born", "Average Household Size", "Count"]
    df_demographics[numeric_columns] = df_demographics[numeric_columns].apply(pd.to_numeric)

    # Convert categorical columns to appropriate data types
    categorical_columns = ["City", "State", "State Code", "Race"]
    df_demographics[categorical_columns] = df_demographics[categorical_columns].astype("category")
    # return the cleaned data:
    return df_demographics

df_demographics = clean_df_demographics(df_demographics)

# Display the cleaned DataFrame
df_demographics.head(10)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712.0,41971.0,80683,4815.0,8355.0,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629.0,56860.0,108489,3800.0,37038.0,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751.0,58077.0,109828,5204.0,16315.0,2.65,NC,Asian,11060


In [47]:
df_demographics.to_csv('./Cleaned Data/df_demographics.csv', index=False)

In [48]:
df_airport_codes = pd.read_csv("./airport-codes_csv.csv")

In [49]:
df_airport_codes.shape

(55075, 12)

In [50]:
df_airport_codes.isnull().sum()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

In [51]:
def clean_df_airport_codes(df_airport_codes):
    # Remove rows with missing values in the 'iata_code' column
    df_airport_codes = df_airport_codes.dropna(subset=['iata_code']).reset_index(drop=True)

    # Convert numeric columns to appropriate data types
    numeric_columns = ["elevation_ft"]
    df_airport_codes[numeric_columns] = df_airport_codes[numeric_columns].apply(pd.to_numeric)

    # Convert categorical columns to appropriate data types
    categorical_columns = ["ident", "type", "name", "continent", "iso_country", "iso_region", "municipality", "gps_code", "iata_code", "local_code"]
    df_airport_codes[categorical_columns] = df_airport_codes[categorical_columns].astype("category")

    # Split the 'coordinates' column into separate 'latitude' and 'longitude' columns
    df_airport_codes[["longitude", "latitude"]] = df_airport_codes["coordinates"].str.split(",", expand=True).apply(pd.to_numeric)

    # filter the  for the United States to make it useful with our immigration data
    df_airport_codes = df_airport_codes[df_airport_codes['iso_country'] == 'US']

    # Drop the original 'coordinates' column
    df_airport_codes = df_airport_codes.drop(columns=["coordinates"])
    # return the cleaned data:
    return df_airport_codes
    
df_airport_codes = clean_df_airport_codes(df_airport_codes)

# Display the cleaned DataFrame
print(df_airport_codes.shape)
df_airport_codes.head(10)

(2019, 13)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,longitude,latitude
1,07FA,small_airport,Ocean Reef Club Airport,8.0,NaN,US,US-FL,Key Largo,07FA,OCA,07FA,-80.274803,25.325399
2,0AK,small_airport,Pilot Station Airport,305.0,NaN,US,US-AK,Pilot Station,NaN,PQS,0AK,-162.899994,61.934601
3,0CO2,small_airport,Crested Butte Airpark,8980.0,NaN,US,US-CO,Crested Butte,0CO2,CSE,0CO2,-106.928341,38.851918
4,0TE7,small_airport,LBJ Ranch Airport,1515.0,NaN,US,US-TX,Johnson City,0TE7,JCY,0TE7,-98.622498,30.251801
5,13MA,small_airport,Metropolitan Airport,418.0,NaN,US,US-MA,Palmer,13MA,PMX,13MA,-72.311401,42.223301
6,13Z,seaplane_base,Loring Seaplane Base,0.0,NaN,US,US-AK,Loring,13Z,WLR,13Z,-131.636993,55.601299
7,16A,small_airport,Nunapitchuk Airport,12.0,NaN,US,US-AK,Nunapitchuk,PPIT,NUP,16A,-162.440454,60.905591
8,16K,seaplane_base,Port Alice Seaplane Base,0.0,NaN,US,US-AK,Port Alice,16K,PTC,16K,-133.597000,55.803000
9,19AK,small_airport,Icy Bay Airport,50.0,NaN,US,US-AK,Icy Bay,19AK,ICY,19AK,-141.662003,59.969002
10,19P,seaplane_base,Port Protection Seaplane Base,0.0,NaN,US,US-AK,Port Protection,19P,PPV,19P,-133.610001,56.328800


In [52]:
df_airport_codes.shape

(2019, 13)

In [53]:
df_airport_codes.isnull().sum()

ident              0
type               0
name               0
elevation_ft      34
continent       2019
iso_country        0
iso_region         0
municipality       6
gps_code          81
iata_code          0
local_code        50
longitude          0
latitude           0
dtype: int64

In [54]:
df_airport_codes = df_airport_codes.drop(columns=["continent"])
df_airport_codes.dropna(inplace=True)

In [55]:
df_airport_codes.head(10)

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,iata_code,local_code,longitude,latitude
1,07FA,small_airport,Ocean Reef Club Airport,8.0,US,US-FL,Key Largo,07FA,OCA,07FA,-80.274803,25.325399
3,0CO2,small_airport,Crested Butte Airpark,8980.0,US,US-CO,Crested Butte,0CO2,CSE,0CO2,-106.928341,38.851918
4,0TE7,small_airport,LBJ Ranch Airport,1515.0,US,US-TX,Johnson City,0TE7,JCY,0TE7,-98.622498,30.251801
5,13MA,small_airport,Metropolitan Airport,418.0,US,US-MA,Palmer,13MA,PMX,13MA,-72.311401,42.223301
6,13Z,seaplane_base,Loring Seaplane Base,0.0,US,US-AK,Loring,13Z,WLR,13Z,-131.636993,55.601299
7,16A,small_airport,Nunapitchuk Airport,12.0,US,US-AK,Nunapitchuk,PPIT,NUP,16A,-162.440454,60.905591
8,16K,seaplane_base,Port Alice Seaplane Base,0.0,US,US-AK,Port Alice,16K,PTC,16K,-133.597000,55.803000
9,19AK,small_airport,Icy Bay Airport,50.0,US,US-AK,Icy Bay,19AK,ICY,19AK,-141.662003,59.969002
10,19P,seaplane_base,Port Protection Seaplane Base,0.0,US,US-AK,Port Protection,19P,PPV,19P,-133.610001,56.328800
11,1KC,small_airport,Kalakaket Creek AS Airport,1598.0,US,US-AK,Kalakaket Creek,1KC,KKK,1KC,-156.820393,64.416626


In [56]:
df_airport_codes.to_csv('./Cleaned Data/df_airport_codes.csv', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

Entities:
- Immigration fact table with columns:
    - cicid (double)
    - year (double)
    - month (double)
    - immigrant_birth_country (double)
    - immigrant_residence_country (double)
    - adm_port (string)
    - arrival_date (double)
    - departure_date (double)
    - arrival_mode (double)
    - arrival_state (string)
    - age (double)
    - visa_type (double)
    - count (double)
    - birth_year (double)
    - gender (string)
    - airline (string)
    - admission_num (double)
    - flight_num (string)
    - visatype (string)
- Temperature dimension table with columns:
    - dt (date)
    - AverageTemperature (double)
    - City (string)
    - Country (string)
- Demographic dimension table with columns:
    - City (string)
    - State (string)
    - median_age (double)
    - male_population (integer)
    - female_population (integer)
    - total_population (integer)
    - num_veterans (integer)
    - num_foreign_born (integer)
    - avg_household_size (double)
    - state_code (string)
    - Race (string)
    - Count (integer)
- Airport dimension table with columns:
    - ident (string)
    - type (string)
    - name (string)
    - elevation_ft (double)
    - iso_country (string)
    - iso_region (string)
    - municipality (string)
    - gps_code (string)
    - iata_code (string)
    - local_code (string)
    - longitude (double)
    - latitude (double)

Relationships:
- The Immigration fact table is related to the Temperature dimension table and the Demographic dimension table through the City and State columns respectively.
- The Immigration fact table is related to the Airport dimension table through the adm_port column representing the airport code.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

***1. Run the create_tables.py script to generate the necessary tables in the database.***

***2. Combine the data from the city and airports tables using a join operation.***

***3. Populate the tables with the relevant data.***

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [57]:
# Create Spark df of all the tables
file_paths = [
    './Cleaned Data/full_immigration.csv',
    './Cleaned Data/df_temp.csv',
    './Cleaned Data/df_demographics.csv',
    './Cleaned Data/df_airport_codes.csv',

]

dataframes = []

for file_path in file_paths:
    df = spark.read.format('csv').option('header', 'true').load(file_path)
    dataframes.append(df)

spark_i94, spark_temp, spark_airport, spark_demographics = dataframes

In [58]:
table_names = ["i94", "temperature", "airport", "demographics"]
for dataframe, name in zip(dataframes, table_names):
    dataframe.createOrReplaceTempView(name)
    dataframe.printSchema()

root
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: string (nullable = 

In [59]:
# Create a SparkSession
spark = SparkSession.builder.appName("immigration_data_pipeline").getOrCreate()

# Load the immigration data
immigration_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load('./Cleaned Data/full_immigration.csv')

# Select relevant columns and rename them from immigration_df:
immigration_fact = immigration_df.select(
    col("cicid"),
    col("i94yr").alias("year"),
    col("i94mon").alias("month"),
    col("i94cit").alias("immigrant_birth_country"),
    col("i94res").alias("immigrant_residence_country"),
    col("i94port").alias("adm_port"),
    col("arrdate").alias("arrival_date"),
    col("depdate").alias("departure_date"),
    col("i94mode").alias("arrival_mode"),
    col("i94addr").alias("arrival_state"),
    col("i94bir").alias("age"),
    col("i94visa").alias("visa_type"),
    col("count"),
    col("biryear").alias("birth_year"),
    col("gender"),
    col("airline"),
    col("admnum").alias("admission_num"),
    col("fltno").alias("flight_num"),
    col("visatype")
)

# Load the temperature data
temperature_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load('./Cleaned Data/df_temp.csv')

# Select relevant columns and filter for data in the United States from temperature_df:
temperature_dim = temperature_df.select(
    col("dt"),
    col("AverageTemperature"),
    col("City"),
    col("Country")
).filter(col("Country") == "United States")

temperature_dim = temperature_dim.dropDuplicates(["dt"])

# Load the demographic data
demographic_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load('./Cleaned Data/df_demographics.csv')

# Select relevant columns and rename them form demographic_df:
demographic_dim = demographic_df.select(
    col("City"),
    col("State"),
    col("Median Age").alias("median_age"),
    col("Male Population").alias("male_population"),
    col("Female Population").alias("female_population"),
    col("Total Population").alias("total_population"),
    col("Number of Veterans").alias("num_veterans"),
    col("Foreign-born").alias("num_foreign_born"),
    col("Average Household Size").alias("avg_household_size"),
    col("State Code").alias("state_code"),
    col("Race"),
    col("Count")
)

demographic_dim = demographic_dim.dropDuplicates(["City", "State"])
demographic_dim = demographic_dim.withColumnRenamed("City", "arrival_city")
# Load the airport data
airport_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load('./Cleaned Data/df_airport_codes.csv')

# Select relevant columns and rename them from airport_df:
airport_schema = StructType([
    StructField("ident", StringType(), True),
    StructField("type", StringType(), True),
    StructField("name", StringType(), True),
    StructField("elevation_ft", DoubleType(), True),
    StructField("iso_country", StringType(), True),
    StructField("iso_region", StringType(), True),
    StructField("municipality", StringType(), True),
    StructField("gps_code", StringType(), True),
    StructField("iata_code", StringType(), True),
    StructField("local_code", StringType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("latitude", DoubleType(), True)
])

airport_dim = airport_df.select(
    col("ident"),
    col("type"),
    col("name"),
    col("iso_country"),
    col("iso_region"),
    col("municipality"),
    col("gps_code"),
    col("iata_code"),
    col("local_code"),
    col("longitude"),
    col("latitude"),
    col("elevation_ft").cast(DoubleType()).alias("elevation_ft")
).filter(col("elevation_ft").isNotNull() & col("elevation_ft").cast("string").rlike("^\\d+\\.?\\d*$"))


In [60]:
immigration_fact.show(10)

+-----+------+-----+-----------------------+---------------------------+--------+------------+--------------+------------+-------------+----+---------+-----+----------+------+-------+--------------+----------+--------+
|cicid|  year|month|immigrant_birth_country|immigrant_residence_country|adm_port|arrival_date|departure_date|arrival_mode|arrival_state| age|visa_type|count|birth_year|gender|airline| admission_num|flight_num|visatype|
+-----+------+-----+-----------------------+---------------------------+--------+------------+--------------+------------+-------------+----+---------+-----+----------+------+-------+--------------+----------+--------+
|  6.0|2016.0|  4.0|                  692.0|                      692.0|     XXX|     20573.0|          null|        null|         null|37.0|      2.0|  1.0|    1979.0|  null|   null| 1.897628485E9|      null|      B2|
|  7.0|2016.0|  4.0|                  254.0|                      276.0|     ATL|     20551.0|          null|         1.0|  

In [61]:
temperature_dim.show(10)

+-------------------+------------------+-------+-------------+
|                 dt|AverageTemperature|   City|      Country|
+-------------------+------------------+-------+-------------+
|1750-03-01 00:00:00|              3.01|  Akron|United States|
|1753-03-01 00:00:00|4.2189999999999985|  Akron|United States|
|1798-04-01 00:00:00|             9.274|  Akron|United States|
|1828-12-01 00:00:00|             7.648|Abilene|United States|
|1854-05-01 00:00:00|            20.789|Abilene|United States|
|1870-01-01 00:00:00| 5.976000000000001|Abilene|United States|
|1879-10-01 00:00:00|            18.047|Abilene|United States|
|1893-11-01 00:00:00|              9.03|Abilene|United States|
|1896-04-01 00:00:00|            18.524|Abilene|United States|
|1902-09-01 00:00:00|21.930999999999997|Abilene|United States|
+-------------------+------------------+-------+-------------+
only showing top 10 rows



In [62]:
demographic_dim.show()

+--------------------+--------------+----------+---------------+-----------------+----------------+------------+----------------+------------------+----------+--------------------+------+
|        arrival_city|         State|median_age|male_population|female_population|total_population|num_veterans|num_foreign_born|avg_household_size|state_code|                Race| Count|
+--------------------+--------------+----------+---------------+-----------------+----------------+------------+----------------+------------------+----------+--------------------+------+
|          Cincinnati|          Ohio|      32.7|       143654.0|         154883.0|          298537|     13699.0|         16896.0|              2.08|        OH|               White|162245|
|         Kansas City|        Kansas|      33.4|        74606.0|          76655.0|          151261|      8139.0|         25507.0|              2.71|        KS|Black or African-...| 40177|
|           Lynchburg|      Virginia|      28.7|        3861

In [63]:
airport_dim.show(10)

+-----+-------------+--------------------+-----------+----------+---------------+--------+---------+----------+-------------------+------------------+------------+
|ident|         type|                name|iso_country|iso_region|   municipality|gps_code|iata_code|local_code|          longitude|          latitude|elevation_ft|
+-----+-------------+--------------------+-----------+----------+---------------+--------+---------+----------+-------------------+------------------+------------+
| 07FA|small_airport|Ocean Reef Club A...|         US|     US-FL|      Key Largo|    07FA|      OCA|      07FA|   -80.274803161621|25.325399398804002|         8.0|
| 0CO2|small_airport|Crested Butte Air...|         US|     US-CO|  Crested Butte|    0CO2|      CSE|      0CO2|-106.92834099999999|         38.851918|      8980.0|
| 0TE7|small_airport|   LBJ Ranch Airport|         US|     US-TX|   Johnson City|    0TE7|      JCY|      0TE7|     -98.6224975586|30.251800537100006|      1515.0|
| 13MA|small_air

In [68]:
"""
creating a query to group count immigrants by their cities and include the "num_foreign_born" field in this city. 
The results could be ordered by the immigrant counts to see if cities with the most immigrants are indeed cities 
with the largest foreign-born counts.
"""
# Join immigration_fact and demographic_dim on arrival_city
joined_df = immigration_fact.join(demographic_dim, immigration_fact.arrival_state == demographic_dim.state_code, "inner")

# Group by arrival_city and sum the immigration_fact.count and demographic_dim.num_foreign_born columns
result_df = joined_df.groupBy("arrival_city") \
    .agg(sum(immigration_fact["count"]).alias("num_immigrants"), sum("num_foreign_born").alias("num_foreign_born")) \
    .orderBy("num_immigrants", ascending=False)

# Show the top 10 cities by immigrant count and their corresponding num_foreign_born count
result_df.show(10)

+--------------+--------------+----------------+
|  arrival_city|num_immigrants|num_foreign_born|
+--------------+--------------+----------------+
|  Jacksonville|      645076.0|  5.333592615E10|
| Pompano Beach|      621701.0| 2.0350138833E10|
|   Tallahassee|      621701.0|  1.039484072E10|
|   Spring Hill|      621701.0|   4.907085993E9|
|Pembroke Pines|      621701.0|  3.867601921E10|
|         Largo|      621701.0|   6.521021789E9|
|         Davie|      621701.0| 1.6494970932E10|
|     Poinciana|      621701.0|   8.920787649E9|
|     Hollywood|      621701.0| 3.4291783758E10|
| Coral Springs|      621701.0| 2.3967816952E10|
+--------------+--------------+----------------+
only showing top 10 rows



In [69]:
result_df.show(100)

+----------------+--------------+----------------+
|    arrival_city|num_immigrants|num_foreign_born|
+----------------+--------------+----------------+
|    Jacksonville|      645076.0|  5.333592615E10|
|      Clearwater|      621701.0| 1.0995403886E10|
|         Deltona|      621701.0|   4.412833698E9|
|        Lakeland|      621701.0|   7.206757992E9|
|         Hialeah|      621701.0|1.05781181748E11|
|      Boca Raton|      621701.0| 1.3128460017E10|
|      Plantation|      621701.0| 1.7346701302E10|
|      Palm Coast|      621701.0|   5.647531884E9|
|Saint Petersburg|      621701.0| 1.7760132467E10|
| West Palm Beach|      621701.0| 1.9070678175E10|
|      Lauderhill|      621701.0| 1.5835346171E10|
|           Davie|      621701.0| 1.6494970932E10|
|           Largo|      621701.0|   6.521021789E9|
|   Coral Springs|      621701.0| 2.3967816952E10|
|         Kendall|      621701.0|  1.938463718E10|
|     Tallahassee|      621701.0|  1.039484072E10|
|           Tampa|      621701.

In [71]:
def write_to_parquet(df, partition_cols, path):
    df.write.mode('overwrite').partitionBy(*partition_cols).parquet(path)
    
# Define the path where the Parquet files will be saved
output_path = "./Transformed Data/"

# Write the immigration_fact data frame to Parquet
write_to_parquet(immigration_fact, ["year", "month"], output_path + "immigration_fact")

# Write the temperature_dim data frame to Parquet
write_to_parquet(temperature_dim, ["City"], output_path + "temperature_dim")

# Write the demographic_dim data frame to Parquet
write_to_parquet(demographic_dim, ["arrival_city", "State"], output_path + "demographic_dim")

# Write the airport_dim data frame to Parquet
write_to_parquet(airport_dim, ["iso_country"], output_path + "airport_dim")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

### immigration_fact Data Quality Check:

In [72]:
def immigration_fact_data_quality_check(immigration_fact):   
    # Check for missing values in the cicid column
    if immigration_fact.filter(col("cicid").isNull()).count() > 0:
        raise ValueError("immigration_fact data frame has missing values in the cicid column")

    # Check for duplicates in the cicid column
    if immigration_fact.count() != immigration_fact.select("cicid").distinct().count():
        raise ValueError("immigration_fact data frame has duplicates in the cicid column")

    # Check that the year and month columns contain valid values
    if immigration_fact.filter(col("year") < 2016).count() > 0:
        raise ValueError("immigration_fact data frame has invalid values in the year column")
    if immigration_fact.filter(col("year") > 2020).count() > 0:
        raise ValueError("immigration_fact data frame has invalid values in the year column")
    if immigration_fact.filter(col("month") < 1).count() > 0:
        raise ValueError("immigration_fact data frame has invalid values in the month column")
    if immigration_fact.filter(col("month") > 12).count() > 0:
        raise ValueError("immigration_fact data frame has invalid values in the month column")
    
immigration_fact_quality_check = immigration_fact_data_quality_check(immigration_fact) 
immigration_fact_quality_check

### temperature_dim Data Quality Check:

In [73]:
def temperature_dim_data_quality_check(temperature_dim):    
    # Check for missing values in the dt column
    if temperature_dim.filter(col("dt").isNull()).count() > 0:
        raise ValueError("temperature_dim data frame has missing values in the dt column")

    # Check for duplicates in the dt column
    if temperature_dim.count() != temperature_dim.select("dt").distinct().count():
        raise ValueError("temperature_dim data frame has duplicates in the dt column")

    # Check that the AverageTemperature column contains valid values
    if temperature_dim.filter(col("AverageTemperature") < -100).count() > 0:
        raise ValueError("temperature_dim data frame has invalid values in the AverageTemperature column")
    if temperature_dim.filter(col("AverageTemperature") > 100).count() > 0:
        raise ValueError("temperature_dim data frame has invalid values in the AverageTemperature column")
           
temperature_dim_quality_check = temperature_dim_data_quality_check(temperature_dim) 
temperature_dim_quality_check

### demographic_dim Data Quality Check:

In [74]:
def demographic_dim_data_quality_check(demographic_dim):
    # Check for missing values in the City and State columns
    if demographic_dim.filter(col("arrival_city").isNull() | col("State").isNull()).count() > 0:
        raise ValueError("demographic_dim data frame has missing values in the City or State column")

    # Check for duplicates in the combination of City and State columns
    if demographic_dim.count() != demographic_dim.select("arrival_city", "State").distinct().count():
        raise ValueError("demographic_dim data frame has duplicates in the combination of City and State columns")

    # Check that the median_age, male_population, female_population, total_population,
    # num_veterans, num_foreign_born, and avg_household_size columns contain valid values
    if demographic_dim.filter(col("median_age") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the median_age column")
    if demographic_dim.filter(col("male_population") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the male_population column")
    if demographic_dim.filter(col("female_population") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the female_population column")
    if demographic_dim.filter(col("total_population") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the total_population column")
    if demographic_dim.filter(col("num_veterans") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the num_veterans column")
    if demographic_dim.filter(col("num_foreign_born") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the num_foreign_born column")
    if demographic_dim.filter(col("avg_household_size") < 0).count() > 0:
        raise ValueError("demographic_dim data frame has invalid values in the avg_household_size column")       
        
demographic_dim_quality_check = demographic_dim_data_quality_check(demographic_dim) 
demographic_dim_quality_check

### airport_dim  Data Quality Check:

In [75]:
def airport_dim_data_quality_check(airport_dim):
    # Check for missing values in the ident column
    if airport_dim.filter(col("ident").isNull()).count() > 0:
        raise ValueError("airport_dim data frame has missing values in the ident column")

    # Check for duplicates in the ident column
    if airport_dim.count() != airport_dim.select("ident").distinct().count():
        raise ValueError("airport_dim data frame has duplicates in the ident column")

    # Check that the elevation_ft, longitude, and latitude columns contain valid values
    if airport_dim.filter(col("elevation_ft").isNull()).count() > 0:
        raise ValueError("airport_dim data frame has missing values in the elevation_ft column")
    if airport_dim.filter(col("longitude").isNull()).count() > 0:
        raise ValueError("airport_dim data frame has missing values in the longitude column")
    if airport_dim.filter(col("latitude").isNull()).count() > 0:
        raise ValueError("airport_dim data frame has missing values in the latitude column")
    if airport_dim.filter(col("elevation_ft") < 0).count() > 0:
        raise ValueError("airport_dim data frame has invalid values in the elevation_ft column")

airport_dim_quality_check = airport_dim_data_quality_check(airport_dim) 
airport_dim_quality_check

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [76]:
tables_dict = {
    "immigration": {
        "immigration_fact": {
            "cicid": "ID number for each record",
            "year": "4 digit year of arrival",
            "month": "Numeric month of arrival",
            "immigrant_birth_country": "3 digit code for immigrant's country of birth",
            "immigrant_residence_country": "3 digit code for immigrant's country of residence",
            "adm_port": "Port of admission into the US",
            "arrival_date": "Arrival date in the US in SAS format",
            "departure_date": "Departure date from the US in SAS format",
            "arrival_mode": "Mode of arrival into the US",
            "arrival_state": "US state of arrival",
            "age": "Age of immigrant",
            "visa_type": "Visa category for immigrant as double type",
            "count": "Number of immigrants with same characteristics",
            "birth_year": "Year of birth for immigrant",
            "gender": "Gender of immigrant",
            "airline": "Airline used to arrive in the US",
            "admission_num": "Admission number for immigrant",
            "flight_num": "Flight number of airline used to arrive in the US",
            "visatype": "Visa category for immigrant as string type"
        }
    },
    "temperature": {
        "temperature_dim": {
            "dt": "Date in format (YYYY-MM-DD) of temperature observation",
            "AverageTemperature": "Average temperature for the city on the given date",
            "City": "Name of the city where the observation was taken",
            "Country": "Country where the city is located"
        }
    },
    "demographics": {
        "demographic_dim": {
            "arrival_city": "Name of the city",
            "State": "US state where the city is located",
            "median_age": "Median age of the population in the city",
            "male_population": "Number of males in the population of the city",
            "female_population": "Number of females in the population of the city",
            "total_population": "Total number of people in the city",
            "num_veterans": "Number of veterans in the population of the city",
            "num_foreign_born": "Number of foreign-born individuals in the population of the city",
            "avg_household_size": "Average household size in the city",
            "state_code": "2 letter code for the US state where the city is located",
            "Race": "Ethnicity of the population",
            "Count": "Number of individuals in the population with the specified ethnicity"
        }
    },
    "airports": {
        "airport_dim": {
            "ident": "Unique identifier for the airport",
            "type": "Type of airport (small, medium, large)",
            "name": "Name of airport",
            "continent": "Continent where airport is located",
            "iso_country": "ISO code for the country where the airport is located",
            "iso_region": "ISO code for the region where the airport is located",
            "municipality": "Municipality where the airport is located",
            "gps_code": "GPS code for the airport",
            "iata_code": "IATA code for the airport",
            "local_code": "Local code for the airport",
             "Latitude and longitude of the airport"
            "longitude":"longitude of the airport",
            "latitude": "Latitude of the airport",
            "elevation_ft": "Airport altitude "
        }
    }
}

In [77]:
# Create an empty list to store the rows of the table
rows = []

# Loop through each table and its nested dictionaries
for table_name, table_dict in tables_dict.items():
    for nested_table_name, nested_table_dict in table_dict.items():
        # Loop through each column and its description in the nested dictionary
        for column_name, column_description in nested_table_dict.items():
            # Add a row to the list for each column and its description
            rows.append([table_name, nested_table_name + '.' + column_name, column_description])

# Create a DataFrame from the list of rows and add column names
database_tables = pd.DataFrame(rows, columns=['Table', 'Column', 'Description'])

# Display the DataFrame
database_tables

,Table,Column,Description
0,immigration,immigration_fact.cicid,ID number for each record
1,immigration,immigration_fact.year,4 digit year of arrival
2,immigration,immigration_fact.month,Numeric month of arrival
3,immigration,immigration_fact.immigrant_birth_country,3 digit code for immigrant's country of birth
4,immigration,immigration_fact.immigrant_residence_country,3 digit code for immigrant's country of residence
5,immigration,immigration_fact.adm_port,Port of admission into the US
6,immigration,immigration_fact.arrival_date,Arrival date in the US in SAS format
7,immigration,immigration_fact.departure_date,Departure date from the US in SAS format
8,immigration,immigration_fact.arrival_mode,Mode of arrival into the US
9,immigration,immigration_fact.arrival_state,US state of arrival


In [78]:
# Stop the SparkSession
spark.stop()

#### Step 5: Complete Project Write Up
 * Clearly state the rationale for the choice of tools and technologies for the project: 
* To handle the massive amount of immigration data, which consists of around 3 million rows, we opted to leverage the power of Apache Spark. For other relatively smaller datasets, we utilized Pandas to manage them.



 * Propose how often the data should be updated and why: 
* Given that the primary dataset (I94 immigration dataset) is updated on a monthly basis, we need to ensure that our solution is updated monthly as well.




 * Write a description of how you would approach the problem differently under the following scenarios:
* The data was increased by 100x: While we will continue to use Spark for handling the big data, we plan to scale up the number of nodes in our cluster. Additionally, we will leverage AWS to store our data in S3 and Redshift.
 
 
 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day: 
* Our ETL pipeline will be managed in a DAG using Apache Airflow, which will guarantee that the pipeline runs on time and performs data quality checks.
 
 
 
 * The database needed to be accessed by 100+ people: 
* To ensure that everyone can easily access our analytics database, we will migrate it to Amazon Redshift.
 
 
 
 * What schema are you using in this project? Does it have any advantage compared to other schemas?: 
* The schema used in this project is a star schema, which comprises of a central fact table (Immigration fact table) connected to several dimension tables (Temperature, Demographic, and Airport dimension tables) through foreign keys.
* The advantage of using a star schema is that it simplifies queries and allows for faster performance, as data is denormalized and stored in a way that minimizes the number of joins required to retrieve data. This schema is also optimized for OLAP (Online Analytical Processing) queries, which involve complex queries that analyze large sets of data.


 * Why do you connect the entities the way you did in this project? Perhaps it is optimized for a particular query?:
* The entities are connected in this way to enable efficient and effective querying of the data, as the foreign keys allow for easy join operations between the fact table and dimension tables. This schema is optimized for queries that involve aggregating data across multiple dimensions, such as analyzing immigration patterns by temperature, demographic, or airport information.